In [1]:
import torch
import os
import pickle
import numpy as np
import itertools
import librosa
import random
from random import shuffle
import codecs
import errno
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision.datasets.mnist
from torchvision import transforms
from tqdm import tqdm
import math

In [2]:
print(os.getcwd())

G:\Git\Deep Learning\Assignment4


In [3]:
# fecthing the data from the pickle files
file = open("hw4_trs.pkl",'rb')
train_data = pickle.load(file)
file.close()

file = open("hw4_tes.pkl",'rb')
test_data = pickle.load(file)
file.close()

In [4]:
train_data.shape
training = []

In [5]:
BATCH = 100

In [6]:
training = torch.zeros(500, 32, 513)
for sig in range(500):
    stft = librosa.stft(train_data[sig], n_fft=1024, hop_length=512)
    training[sig] = torch.tensor(np.abs(stft).T)

In [7]:
positive_list = []
negative_list = []
take_samples = 45
training_person_count = 50
testing_person_count = 20

In [8]:
def get_pairs(number_of_speakers):
    positive_pairs = np.zeros((number_of_speakers,take_samples,3))
    negative_pairs = np.zeros((number_of_speakers,take_samples,3))
    for i in range(number_of_speakers):
        complete_set = set(range(number_of_speakers * 10))
        positive_set = set(range((i * 10), ((i *10)+ 10)))
        negative_set = set(complete_set - positive_set)
        positive_list = list(positive_set)
        negative_list = list(negative_set)
        positive_combinations = np.array(random.sample(list(itertools.combinations(positive_list, 2)), take_samples) )
        negative_combinations = np.array(random.sample(list(itertools.product(positive_list, negative_list)), take_samples))
        positive_labels = np.ones((take_samples,1))
        negative_labels = np.zeros((take_samples,1))
        pos = np.hstack((positive_combinations, positive_labels))
        neg = np.hstack((negative_combinations, negative_labels))
        positive_pairs[i] = pos
        negative_pairs[i] = neg
    return positive_pairs, negative_pairs

In [9]:
positive_pairs, negative_pairs = get_pairs(50)

In [10]:
print(positive_pairs.shape)

(50, 45, 3)


In [11]:
print(training[0].shape)

torch.Size([32, 513])


In [12]:
positive_pairs_reshaped = positive_pairs.reshape((training_person_count * take_samples), 3)
negative_pairs_reshaped = negative_pairs.reshape((training_person_count * take_samples), 3)

In [13]:
data = np.concatenate((positive_pairs_reshaped, negative_pairs_reshaped))
data = np.array(data, dtype=np.int)
print(data.shape)


(4500, 3)


In [14]:
train_loader = torch.utils.data.DataLoader(data, shuffle=True, batch_size=BATCH)
print(len(train_loader))

45


In [15]:
# Ref : https://github.com/adambielski/siamese-triplet/blob/master/losses.py
# ref: https://hackernoon.com/one-shot-learning-with-siamese-networks-in-pytorch-8ddaab10340e
class ContrastiveLoss(nn.Module):
    """
    Contrastive loss
    Takes embeddings of two samples and a target label == 1 if samples are from the same class and label == 0 otherwise
    """

    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(1)  # squared distances
        losses = 0.5 * (target.float() * distances +
                        (1 + -1 * target).float() * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2))
        return losses.mean() if size_average else losses.sum()

In [ ]:
# https://hackernoon.com/one-shot-learning-with-siamese-networks-in-pytorch-8ddaab10340e
# ref: https://becominghuman.ai/siamese-networks-algorithm-applications-and-pytorch-implementation-4ffa3304c18
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, 2)
        self.linear1 = nn.Linear(1600, 800)
        self.linear2 = nn.Linear(800, 400)
        self.linear3 = nn.Linear(400, 50)
        self.linear4 = nn.Linear(50, 20)
        
        self.act = F.relu
        
    def siamese_forward(self, x):
        pred, hidden = self.rnn(x, None)
        pred = pred.view(pred.shape[0], -1)
        pred = self.act(self.linear1(pred))
        pred = self.act(self.linear2(pred))
        pred = self.act(self.linear3(pred))
        pred = self.act(self.linear4(pred))
        return pred
    
    def forward(self, x , y):
        first = self.siamese_forward(x)
        second = self.siamese_forward(y)
        return first,second

neural_network = LSTM(513,50)
neural_network = neural_network.cuda()
loss_function= ContrastiveLoss(2)
para = neural_network.parameters()
optimizer = torch.optim.Adam(params=para, lr=0.0003)
loss_list = []
epochs = 150
for i in range(epochs):
    train_loader_iter = iter(train_loader)
    while True:
        try:
            pairs_label = train_loader_iter.next()
            optimizer.zero_grad()
            first_set = training[pairs_label[:,0].numpy()].cuda()
            second_set = training[pairs_label[:,1].numpy()].cuda()
            labels = pairs_label[:,2].cuda().float() 
            
            first,second = neural_network(first_set, second_set)
            loss = loss_function(first,second, labels)
            loss.backward()
            optimizer.step()
            
        except StopIteration:
            break
    print(loss)
    loss_list.append(loss.data.cpu().numpy())
print(loss.data)
plt.plot(range(epochs), loss_list)
plt.show() 

tensor(0.6181, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.5548, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.4973, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.4089, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.2490, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.2377, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.1952, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.1523, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.1408, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.1419, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.1201, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0896, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0667, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0718, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0698, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0635, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0478, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.0461, device='cuda:0',

In [ ]:
positive_pairs, negative_pairs = get_pairs(20)

In [ ]:
positive_pairs.shape

In [ ]:
positive_pairs_reshaped = positive_pairs.reshape((testing_person_count * take_samples), 3)
negative_pairs_reshaped = negative_pairs.reshape((testing_person_count * take_samples), 3)

data_test = np.concatenate((positive_pairs_reshaped, negative_pairs_reshaped))
data_test = np.array(data_test, dtype=np.int)
print(data_test.shape)

In [ ]:
test_pair_loader = torch.utils.data.DataLoader(data_test, shuffle=True, batch_size=1)


In [ ]:
testing = torch.zeros(500, 45, 513)
for sig in range(200):
    stft = librosa.stft(test_data[sig], n_fft=1024, hop_length=512)
    testing[sig] = torch.tensor(np.abs(stft).T)

In [ ]:
testing = testing[:,0:32,:]
testing.shape

In [ ]:
correct = 0
with torch.no_grad():
    pairs_iter = iter(test_pair_loader)
    while True:
        try:
            pairs_label = pairs_iter.next()
            optimizer.zero_grad()
            first_set = testing[pairs_label[:,0].numpy()].cuda()
            second_set = testing[pairs_label[:,1].numpy()].cuda()
            labels = pairs_label[:,2].cuda().float()
            first_output, second_output = neural_network(first_set, second_set)
            
            distance = F.pairwise_distance(first_output, second_output).cpu().numpy()[0]

            if distance < 3.1:
                predicted = 1
            else:
                predicted = 0
            if predicted == labels[0].cpu().numpy():
                correct = correct + 1
            
        except StopIteration:
            print(correct/18)
            break 